In [25]:
import numpy as np
import time, os

class HopfieldNetwork:
    def __init__(self, patterns, dtype=np.float):
        '''
        a discrete Hopfield network.\n
        patterns: a non-empty container which contains the patterns to be stored in the network.
        each pattern is a numpy array of int [1, -1], each of the same length
        '''
        input_size = patterns[0].shape[0]
        self.matrix = np.zeros((input_size, input_size), dtype=dtype)
        for p in patterns:
            self.matrix += np.outer(p, p)
        np.fill_diagonal(self.matrix, 0)
    
    def get_synchronous(self, pattern):
        '''
        modifies the input, attempting to retrieve the most similar stored pattern.\n
        all elements are updated at the same time
        '''
        output_write = pattern
        while True:
            mat_out = np.matmul(self.matrix, pattern)
            new_pattern = np.ones(mat_out.shape, dtype=np.int)
            np.copysign(new_pattern, mat_out)
            change_occured = np.array_equal(new_pattern, pattern)
            if change_occured:
                break
            pattern = new_pattern
        output_write[:] = new_pattern

    def get_stochastic(self, pattern, seed=(int(time.time()) * os.getpid()) & 0x7FFFFFFF):
        '''
        modifies the input, attempting to retrieve the most similar stored pattern.\n
        the update order is deterministic, random, and cyclic.
        '''
        random_state = np.random.RandomState(seed)
        while True:
            candidate_indices = [i for i in range(self.matrix.shape[0])]
            random_state.shuffle(candidate_indices)
            change_occured = False
            while len(candidate_indices) != 0:
                candidate_indices_index = random_state.randint(0, len(candidate_indices))
                index = candidate_indices[candidate_indices_index]
                del candidate_indices[candidate_indices_index]

                next_at_index = 1 if np.positive(np.dot(self.matrix[index], pattern)) else -1
                if next_at_index != pattern[index]:
                    change_occured = True
                pattern[index] = next_at_index
            if not change_occured:
                break


In [26]:
pattern_1 = np.array([1,  1,  1,  1,  1,  1])
pattern_2 = np.array([1, -1,  1, -1,  1, -1])
patterns = [pattern_1, pattern_2]
h = HopfieldNetwork(patterns)

retrieval = [1,  1,  1,  1,  1, -1]
h.get_synchronous(retrieval)
print(retrieval)


[1, 1, 1, 1, 1, 1]
